In [ ]:
import intake
import xarray as xr

df = intake.open_csv("./data/bird_migration/{species}.csv").read()
air = xr.open_dataset("./data/air.nc").air
air = air.sel(lon=slice(205, 310), lat=slice(75, -55))

## Panel 

In addition to widgets, `panel` provides the framework and interactivity to make dashboards that work in the notebook and can be deployed as standalone apps. In this notebook we'll take the the pieces from that we've learned about so far (`hvplot`, `interactive`, `panel`) and put it all together into a dashboard.

In [ ]:
import panel as pn

pn.extension("tabulator")

import hvplot.pandas
import hvplot.xarray

### The widgets

In [ ]:
day_slider = pn.widgets.IntSlider(name="day", start=1, end=365, step=7)
species_selector = pn.widgets.Select(
    name="species", options=sorted(list(df.species.cat.categories))
)

### Map of the birds grouped by species and day

In [ ]:
bird_map = (
    air.hvplot(
        "lon",
        "lat",
        groupby="day",
        geo=True,
        cmap="coolwarm",
        clim=(-40, 40),
        coastline=True,
        height=600,
    )
    * df.hvplot.points(
        "lon",
        "lat",
        groupby="day",
        hover_cols=["species"],
        geo=True,
        color="white",
    )
    * df.hvplot.points(
        "lon",
        "lat",
        groupby=["day", "species"],
        hover=False,
        geo=True,
        color="black",
        size=200,
        title="Bird Location and Air Temperature (C)",
    )
)

### Overview of the year for a particular bird

In [ ]:
bird_lon_lat = df.hvplot.points(
    "lon",
    "lat",
    color="day",
    cmap="colorwheel",
    groupby="species",
    title="",
    colorbar=False,
)
bird_day_lat = df.hvplot.hist(
    "lat",
    groupby="species",
    title="",
)

This is setting up the title of the overview section. I'm not going to go into it in this talk but you can learn all about the different ways in which you can link bits together in the [Panel Linking Docs](https://panel.holoviz.org/user_guide/Links.html).

In [ ]:
def callback(target, event):
    target.object = f"### Overview of year for: {event.new.replace('_', ' ')}"


title = pn.pane.Markdown(
    f"### Overview of year for: {species_selector.value.replace('_', ' ')}"
)
species_selector.link(title, callbacks={"value": callback})

### Table of nearby birds

In [ ]:
df_i = df.interactive()

lon, lat = df_i[(df_i.species == species_selector) & (df_i.day == day_slider)][
    ["lon", "lat"]
].values[0]

pipeline = (
    df_i[
        (df_i.day == day_slider)
        & (df_i.lon > lon - 5)
        & (df_i.lon < lon + 5)
        & (df_i.lat > lat - 5)
        & (df_i.lat < lat + 5)
    ]
    .set_index(["lon", "lat"])
    .sort_index()[["species"]]
)

### Explanatory text

In [ ]:
description = """
Explore bird data collected by citizen scientists and cleaned and compiled by
scientists. This data tracks the migratory patterns of various bird species
over the course of a typical year. The data are coordinates of the centroid of
different bird species with for each day of the year.
"""
reference_1 = """
La Sorte FA, Fink D, Hochachka WM, Kelling S. 2016 "Convergence of broad-scale
migration strategies in terrestrial birds". Proc. R. Soc. B 283: 20152588.
http://dx.doi.org/10.1098/rspb.2015.2588
"""
reference_2 = """
La Sorte FA, and Fink D. 2017 "Projected changes in prevailing winds for
transatlantic migratory birds under global warming".
Journal of American Ecology 2017.86: 273-284.
doi: 10.1111/1365-2656.12624
"""

### Set up the dashboard using a template

In [ ]:
# Layout using Template
template = pn.template.FastListTemplate(
    title="Bird Migration",
    sidebar=[
        pn.pane.Markdown("# Bird Migration"),
        pn.pane.Markdown(description),
        pn.pane.Markdown("## Settings"),
        species_selector,
        day_slider,
        pn.pane.Markdown("## References"),
        pn.pane.Markdown(reference_1),
        pn.pane.Markdown(reference_2),
    ],
    main=[
        pn.Row(
            pn.Column(
                pn.Pane(
                    bird_map, widgets={"species": species_selector, "day": day_slider}
                ),
            ),
            pn.Column(
                title,
                pn.Tabs(
                    pn.Pane(
                        bird_lon_lat,
                        widgets={"species": species_selector},
                        name="Location on every day",
                    ),
                    pn.Pane(
                        bird_day_lat,
                        widgets={"species": species_selector},
                        name="Days spent at various lats",
                    ),
                    margin=(0, 10),
                ),
                pn.Row(
                    pn.Column(
                        pn.pane.Markdown("### Nearby Birds"),
                        pn.pane.Markdown(
                            "Birds that were within 5 degrees lat and 5 degrees "
                            "lon of the selected bird on the selected day of the year"
                        ),
                        width=200,
                    ),
                    pipeline.pipe(
                        pn.widgets.Tabulator,
                        pagination="remote",
                        page_size=10,
                        width=400,
                    ).panel(),
                ),
                width=600,
                margin=(0, 30),
            ),
        )
    ],
    accent_base_color="#0479cc",
    header_background="#0479cc",
)
template.show()